# Description

Continuing the theme of the last two lesson exercises, we again work with a collection of 1000 files, each of which contain 20 integers, one per line.  As in the earlier exercise, the threaded style on these small files is unlikely to prove faster than a purely serial approach (but with larger files or slower reads, the balance co

# Setup



In [1]:
from threading import Thread, Lock

# Solution

1

# Test Cases